## Notebook Summary
#### *Capstone: Data Cleaning #2*
---
This contents of this notebook includes the data cleaning of the datasets for each year after all the features were merged together in the previous data cleaning notebook

### Datasets
---

There are datasets included in the [`datasets`](./datasets/) 

* [`18-20-Report-Card-Public-Data-Set.xlsx`](../Capstone/datasets/18-20-Report-Card-Public-Data-Set.xlsx): school academic, demographic, race, and other descriptions
* [`18-21-Financial.xlsx`](../Capstone/datasets/18-21-Financial.xlsx): school and district financial data
* [`EDGE-18-20-Poverty-Data.xlsx`](../Capstone/datasets/EDGE-18-20-Poverty-Data.xlsx): school income to poverty data estimates

In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pickle

pd.set_option('display.max_rows', None)
# pd.set_option('max_colwidth', 150)

import warnings
warnings.filterwarnings('ignore')

#### Read in datasets

In [2]:
#reading in dataset was deleting leading zeros so need to adjust read in

# column names which need to be string
cols = ['RCDTS', 'NCES ID']
dict_dtypes = {x: 'str' for x in cols}

sy18 = pd.read_csv('../Capstone/cleaned_datasets/cleaning/df_18.csv', dtype=dict_dtypes)
sy19 = pd.read_csv('../Capstone/cleaned_datasets/cleaning/df_19.csv', dtype=dict_dtypes)
sy20 = pd.read_csv('../Capstone/cleaned_datasets/cleaning/df_20.csv', dtype=dict_dtypes)

In [3]:
print(sy18.shape)
print(sy19.shape)
print(sy20.shape)

(3797, 397)
(3801, 858)
(3814, 915)


### Create function to pre-clean all 3 datasets
- dropping PreK because it's a small % of dataset and don't think it's as valuable in terms of discipline data
- I am going to build 2 different models - HS vs Elem/Middle because there are a lot of elementary schools that serve up to 8th grade.
- There are several schools serving all grades so I am including those schools in both final datasets for modeling
- For charter, I am creating a new col - charter and then organize them by High School, Lower School, and All Grades based on the grades served

In [4]:
def clean(df): #input is the df  

    # drop columns I know I don't need across all datasets
    dropcols = ['Unnamed: 0', 'District Name', 'School', 'Avg Class Size - Kindergarten', 'Avg Class Size - 1',
                'Avg Class Size - 2','Avg Class Size - 3','Avg Class Size - 4','Avg Class Size - 5','Avg Class Size - 6',
                'Avg Class Size - 7', 'Avg Class Size - 8', 'Avg Class Size - High School', 'District Type', 'District Size']
    
    df.drop(columns=dropcols, inplace=True)

    # drop pre-K from dataset
    df = df[df['School Type'] != 'PreK']
    
    # dropping rows where 'School Type' is null
    df = df.dropna(subset=['School Type'])
    
    # create a new col grouping schools in High School, Lower School, or Charter
    df['school_type2'] = ['High School' if x == 'HIGH SCHOOL' else 'Lower School' if (x == 'ELEMENTARY' or x == 'MIDDLE SCHL') else 'Charter' for x in df['School Type']]
    
    # create new col charter Y/N
    df['Charter'] = ['Y' if x == 'Charter' else 'N' for x in df['school_type2']]
    
    # assigning charter schools to lower/high school
    # list of grades served and assigned 
    hs = [' 9 10 11 12', ' 6 7 8 9 10 11 12', ' 7 8 9 10 11 12', ' 9 10', ' 9 10 11', ' 6 7 8 9 10', ' 8 9 10 11 12', ' 11 12']
    allgrades = [' K 1 2 3 4 5 6 7 8 9 10 11 12', ' 3 4 5 6 7 8 9 10 11 12', ' K 1 2 3 4 5 6 7 8 9 10 11', ' 5 6 7 8 9 10 11 12', ' 4 5 6 7 8 9 10']
    
    # if it's a charter and it meets these 2 conditions, change to High School
    for g in hs:
        df['school_type2'] = np.where(
            (df['Grades Served'] == g) & (df['School Type'] == 'CHARTER SCH'), 'High School', df['school_type2'])
    
    # if it's a charter and it meets these 2 conditions, change to All grades   
    for g in allgrades:
        df['school_type2'] = np.where(
            (df['Grades Served'] == g) & (df['School Type'] == 'CHARTER SCH'), 'All Grades', df['school_type2']
            )
    
    # if it's a charter and it meets these 2 conditions, change to Lower School
    df['school_type2'] = np.where(
        (df['school_type2'] == 'Charter') & (df['School Type'] == 'CHARTER SCH'), 'Lower School', df['school_type2'])
    
    # drop columns that are completely null
    df = df.dropna(axis=1, how='all')
    
    # rename columns to match across datasets
    cols_to_rename = {'Min per Day Scieince Grade 3' : 'Min per Day Science Grade 3',
                      'Min per Day Scieince Grade 6' : 'Min per Day Science Grade 6',
                      'Min per Day Scieince Grade 8' : 'Min per Day Science Grade 8',
                      'Min per Day social sciences Grade 3': 'Min per Day Social Sciences Grade 3',
                      'Min per Day social sciences Grade 6': 'Min per Day Social Sciences Grade 6',
                      'Min per Day social sciences Grade 8': 'Min per Day Social Sciences Grade 8',
                      '# Student Enrollment': 'Student Enrollment - Total',
                      'Student Enrollment - White %': '% Student Enrollment - White',
                      'Student Enrollment - Black or African American %': '% Student Enrollment - Black or African American',
                      'Student Enrollment - Hispanic or Latino %': '% Student Enrollment - Hispanic or Latino',
                      'Student Enrollment - Asian %': '% Student Enrollment - Asian',
                      'Student Enrollment - Native Hawaiian or Other Pacific Islander %': '% Student Enrollment - Native Hawaiian or Other Pacific Islander',
                      'Student Enrollment - American Indian or Alaska Native %': '% Student Enrollment - American Indian or Alaska Native',
                      'Student Enrollment - Two or More Races %': '% Student Enrollment - Two or More Races',
                      'Student Enrollment - EL %' : '% Student Enrollment - EL',
                      'Student Enrollment - IEP %': '% Student Enrollment - IEP' ,
                      'Student Enrollment - Low Income %': '% Student Enrollment - Low Income',
                      'Student Enrollment - Homeless %': '% Student Enrollment - Homeless'
                     }
    
    df.rename(columns = cols_to_rename, inplace=True)
    
    
    return df

In [5]:
#run all datasets through cleaning function and reassign to df
sy18 = clean(sy18)
sy19 = clean(sy19)
sy20 = clean(sy20)

### Drop specific columns from the 2018 dataset

In [6]:
colsdrop18 = ['Student Attendance Rate - Male',
       'Student Attendance Rate - Female', 'Student Attendance Rate - White',
       'Student Attendance Rate - Black or African American',
       'Student Attendance Rate - Hispanic or Latino',
       'Student Attendance Rate - Asian',
       'Student Attendance Rate - Native Hawaiian or Other Pacific Islander',
       'Student Attendance Rate - American Indian or Alaska Native',
       'Student Attendance Rate - Two or More Races',
       'Student Attendance Rate - EL','%8th Grade passing Algebra 1', 'Chronic Absenteeism']

sy18.drop(columns=colsdrop18, inplace=True)

In [7]:
print(sy18.shape)
print(sy19.shape)
print(sy20.shape)

(3709, 294)
(3710, 365)
(3730, 392)


### Drop specific columns from the 2019 & 2020 dataset

In [8]:
colsdrop18 = ['% Student Enrollment - Children with Disabilities',
              'Chronic Absenteeism - Children with Disabilities',
              'Chronic Absenteeism - Female',
              'Chronic Absenteeism - Male',
              'Student Mobility Rate - Children with Disabilities',
              'Title 1 Status', '% 8th Grade Passing Algebra 1', 'Chronic Absenteeism']

sy19.drop(columns=colsdrop18, inplace=True)
sy20.drop(columns=colsdrop18, inplace=True)

In [9]:
print(sy18.shape)
print(sy19.shape)
print(sy20.shape)

(3709, 294)
(3710, 357)
(3730, 384)


### Drop additional columns from All Datasets

In [10]:
# dropping additional data cols from all datasets

drop = ['Five Essential', 'CTE', 'Physical Education', 'Summative Designation: Student Group(s)', 'Giftedness', 'Gifted', 'Min per Day']

def drop_cols(df):
    for x in drop:
        drop_list = df.filter(like=x).columns
        df.drop(columns=drop_list, inplace=True)
    
    return df

In [11]:
sy18_clean = drop_cols(sy18)
sy19_clean = drop_cols(sy19)
sy20_clean = drop_cols(sy20)

### Split dataframes into High School & Lower School DataFrames
---

In [12]:
sy18_hs = sy18_clean[(sy18_clean['school_type2'] == 'High School') | (sy18_clean['school_type2'] == 'All Grades')]
sy18_lower = sy18_clean[(sy18_clean['school_type2'] == 'Lower School') | (sy18_clean['school_type2'] == 'All Grades')]
sy19_hs = sy19_clean[(sy19_clean['school_type2'] == 'High School') | (sy19_clean['school_type2'] == 'All Grades')]
sy19_lower = sy19_clean[(sy19_clean['school_type2'] == 'Lower School') | (sy19_clean['school_type2'] == 'All Grades')]
sy20_hs = sy20_clean[(sy20_clean['school_type2'] == 'High School') | (sy20_clean['school_type2'] == 'All Grades')]
sy20_lower = sy20_clean[(sy20_clean['school_type2'] == 'Lower School') | (sy20_clean['school_type2'] == 'All Grades')]

In [13]:
print(sy18_hs.shape)
print(sy19_hs.shape)
print(sy20_hs.shape)
print(sy18_lower.shape)
print(sy19_lower.shape)
print(sy20_lower.shape)

(709, 268)
(710, 330)
(714, 337)
(3008, 268)
(3009, 330)
(3025, 337)


## Data Cleaning Lower School DFs

#### Drop all HS related columns

In [14]:
#list of hs key words to search for
drop = ['High School', '9th Grade', 'AP', 'IB classes', 'Dual Credit', 'Post Secondary', 
        'advanced Courses', 'Community College', 'CRDC', 'Advanced Courses', 'Postsecondary Institution']

def lower_clean(df):
    for x in drop:
        drop_list = df.filter(like=x).columns
        df.drop(columns=drop_list, inplace=True)
    
    return df

In [15]:
# apply to lower school dfs
sy18_lower = lower_clean(sy18_lower)
sy19_lower = lower_clean(sy19_lower)
sy20_lower = lower_clean(sy20_lower)

In [16]:
print(sy18_lower.shape)
print(sy19_lower.shape)
print(sy20_lower.shape)

(3008, 59)
(3009, 59)
(3025, 59)


In [17]:
# Double checking that columns match
#sy18_lower.columns.difference(sy19_lower.columns)

---

### Read in ELA/Math Report Card Dataset from State of IL

In [18]:
scores_18 = pd.read_excel('datasets/18-20-Report-Card-Public-Data-Set.xlsx', sheet_name = '17-18-ELA and Math', converters={'RCDTS': str})
scores_19 = pd.read_excel('datasets/18-20-Report-Card-Public-Data-Set.xlsx', sheet_name = '18-19-ELA and Math', converters={'RCDTS': str})
scores_20 = pd.read_excel('datasets/18-20-Report-Card-Public-Data-Set.xlsx', sheet_name = '19-20-ELA and Math', converters={'RCDTS': str})

In [19]:
print(scores_18.shape)
print(scores_19.shape)
print(scores_20.shape)

(4754, 160)
(4738, 236)
(4727, 21)


In [20]:
scores_18.head()

,RCDTS,Type,School Name,District,City,County,District Type,District Size,School Type,Grades Served,...,Science Grade 5 Proficient,ELA Grade 6 Proficient,Math Grade 6 Proficient,ELA Grade 7 Proficient,Math Grade 7 Proficient,ELA Grade 8 Proficient,Math Grade 8 Proficient,Science Grade 8 Proficient,ELA Grade 11 Proficient,Math Grade 11 Proficient
0,010010010260000,District,NaN,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,NaN,NaN,...,22.0,15.0,8.0,14.0,13.0,12.0,6.0,23.0,15.0,11.0
1,010010010260001,School,Seymour High School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,NaN,NaN,NaN,14.0,13.0,12.0,6.0,23.0,15.0,11.0
2,010010010262002,School,Seymour Elementary School,Payson CUSD 1,Payson,Adams,UNIT,MEDIUM,ELEMENTARY,PK K 1 2 3 4 5 6,...,22.0,15.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010010020260000,District,NaN,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,NaN,NaN,...,21.0,13.0,14.0,22.0,17.0,17.0,15.0,33.0,17.0,12.0
4,010010020260001,School,Liberty High School,Liberty CUSD 2,Liberty,Adams,UNIT,MEDIUM,HIGH SCHOOL,7 8 9 10 11 12,...,NaN,NaN,NaN,22.0,17.0,17.0,15.0,33.0,17.0,12.0


In [21]:
#filter rows where type = School
scores_18 = scores_18[scores_18['Type'] == 'School']
scores_19 = scores_19[scores_19['Type'] == 'School']
scores_20 = scores_20[scores_20['Type'] == 'School']

In [22]:
#only selecting columns needed
assessment_18 = scores_18[['RCDTS', 'ELA Proficiency Total %', 'ELA Proficiency Low Income %', 'Math Proficiency Total %', 'Math Proficiency Low Income %']]
assessment_19 = scores_19[['RCDTS', '% ELA Proficiency', '% ELA Proficiency - Low Income', '% Math Proficiency', '% Math Proficiency - Low Income']]

#no assessment data for 2020 available. Can I still use this for the other years?

In [23]:
#rename columns to match
assessment_18.rename(columns = {'ELA Proficiency Total %': '% ELA Proficiency',
                                'ELA Proficiency Low Income %': '% ELA Proficiency - Low Income',
                                'Math Proficiency Total %': '% Math Proficiency',
                                'Math Proficiency Low Income %': '% Math Proficiency - Low Income'}, inplace=True)

In [24]:
# since 2020 was the year covid started there's no assessment data scores for that year, but this is a feature I want to use in the model
# going to use time series logic of last data to predict the 2019-2020 data by copying 2018-2019 year's data over

#grab just the school code from the raw dataset
assessment_20 = scores_20[['RCDTS']]

#grab the scores from the previous year and add it to the 20 dataset
assessment_20 = assessment_20.merge(assessment_19, how='left', on = 'RCDTS')

## Source: NCES Data Cleaning
---

### Read in school income-to-poverty estimates dataset from NCES

In [25]:
# read in the data from all 3 sheets
incometopov_18 = pd.read_excel('datasets/EDGE-18-20-Poverty-Data.xlsx', sheet_name = '17-18', converters={'NCESSCH': str})
incometopov_19 = pd.read_excel('datasets/EDGE-18-20-Poverty-Data.xlsx', sheet_name = '18-19', converters={'NCESSCH': str})
incometopov_20 = pd.read_excel('datasets/EDGE-18-20-Poverty-Data.xlsx', sheet_name = '19-20', converters={'NCESSCH': str})

In [26]:
#only select the columns I need for merge
ipr_18 = incometopov_18[['NCESSCH', 'IPR_EST']]
ipr_19 = incometopov_19[['NCESSCH', 'IPR_EST']]
ipr_20 = incometopov_20[['NCESSCH', 'IPR_EST']]

### Read in school characteristics dataset from NCES

In [27]:
#read in the data from all 3 sheets
# nces18 = pd.read_excel('datasets/18-20-Public_School_Characteristics.xlsx', sheet_name = '17-18', converters={'NCESSCH': str})
# nces19 = pd.read_excel('datasets/18-20-Public_School_Characteristics.xlsx', sheet_name = '18-19', converters={'NCESSCH': str})
# nces20 = pd.read_excel('datasets/18-20-Public_School_Characteristics.xlsx', sheet_name = '19-20', converters={'NCESSCH': str})

In [28]:
# nces18 = nces18[nces18['STABR'] == 'IL']
# nces19 = nces19[nces19['STABR'] == 'IL']
# nces20 = nces20[nces20['STABR'] == 'IL']

In [29]:
#only select the columns I need for merge
# nces18 = nces18[['NCESSCH', 'STITLEI', 'STUTERATIO', 'ULOCALE', 'FTE', 'LATCOD', 'LONCOD']]
# nces19 = nces19[['NCESSCH', 'STITLEI', 'STUTERATIO', 'ULOCALE', 'FTE', 'LATCOD', 'LONCOD']]
# nces20 = nces20[['NCESSCH', 'STITLEI', 'STUTERATIO', 'ULOCALE', 'FTE', 'LATCOD', 'LONCOD']]

In [30]:
#pickle these dfs so I don't need to rerun the original dataset
#commenting out so I don't run again

# nces18.to_pickle('../Capstone/pickles/nces18.pkl')
# nces19.to_pickle('../Capstone/pickles/nces19.pkl')
# nces20.to_pickle('../Capstone/pickles/nces20.pkl')

In [31]:
#load in dfs
nces18 = pd.read_pickle('../Capstone/pickles/nces18.pkl')
nces19 = pd.read_pickle('../Capstone/pickles/nces19.pkl')
nces20 = pd.read_pickle('../Capstone/pickles/nces20.pkl')

## Merge NCES datasets & IL Assessment dataset with Lower School 18-20 Datasets

In [32]:
print(sy18_lower.shape)
print(sy19_lower.shape)
print(sy20_lower.shape)

(3008, 59)
(3009, 59)
(3025, 59)


In [33]:
# NCES ID
# NCESSCH

In [34]:
# merge with school characteristics
sy18_low_final = sy18_lower.merge(nces18, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')
sy19_low_final = sy19_lower.merge(nces18, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')
sy20_low_final = sy20_lower.merge(nces18, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')

#drop NCESSCH col
sy18_low_final.drop(columns = 'NCESSCH', inplace=True)
sy19_low_final.drop(columns = 'NCESSCH', inplace=True)
sy20_low_final.drop(columns = 'NCESSCH', inplace=True)

In [35]:
# merge with IPR data
sy18_low_final = sy18_low_final.merge(ipr_18, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')
sy19_low_final = sy19_low_final.merge(ipr_19, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')
sy20_low_final = sy20_low_final.merge(ipr_20, how = 'left', left_on = 'NCES ID', right_on = 'NCESSCH')

#drop NCESSCH col
sy18_low_final.drop(columns = 'NCESSCH', inplace=True)
sy19_low_final.drop(columns = 'NCESSCH', inplace=True)
sy20_low_final.drop(columns = 'NCESSCH', inplace=True)

In [36]:
# merge with assessment data
sy18_low_final = sy18_low_final.merge(assessment_18, how = 'left', on = 'RCDTS')
sy19_low_final = sy19_low_final.merge(assessment_19, how = 'left', on = 'RCDTS')
sy20_low_final = sy20_low_final.merge(assessment_20, how = 'left', on = 'RCDTS')

### Merging in Financial Data for each SY

In [37]:
#each represent what percentage of school funding came from which stream
#so for 2018 SY data - I should pull from 

In [38]:
#financial data for the previous school year is available the following year
#read in data
financial18 = pd.read_excel('../Capstone/datasets/18-21-Financial.xlsx', sheet_name='18-19-Financial', converters={'RCDTS': str})

#filter for district only
financial18 = financial18[financial18['Type'] == 'District']

#select specific columns
financial18 = financial18[['District','District Size',
                '% Local Property Taxes 2017-18', '% General State Aid 2017-18',
                '% Federal Funding 2017-18', '$ Instructional Expenditure per Pupil 2017-18']]

In [39]:
#financial data for the previous school year is available the following year
#read in data
financial19 = pd.read_excel('../Capstone/datasets/18-21-Financial.xlsx', sheet_name='19-20-Financial', converters={'RCDTS': str})

#filter for district only
financial19 = financial19[financial19['Type'] == 'District']

#select specific columns
financial19 = financial19[['District', 'District Size',
                '% Local Property Taxes', '% General State Aid',
                '% Federal Funding', '$ Instructional Expenditure per Pupil']]

In [40]:
#financial data for the previous school year is available the following year
#read in data
financial20 = pd.read_excel('../Capstone/datasets/18-21-Financial.xlsx', sheet_name='20-21-Financial', converters={'RCDTS': str})

#filter for district only
financial20 = financial20[financial20['Type'] == 'District']

#select specific columns
financial20 = financial20[['District', 'District Size',
                '% Local Property Taxes', '% General State Aid',
                '% Federal Funding', '$ Instructional Expenditure per Pupil']]

In [41]:
#renaming column headers to match
financial18.rename(columns = {'% Local Property Taxes 2017-18': '% Local Property Taxes',
                    '% General State Aid 2017-18': '% General State Aid',
                    '% Federal Funding 2017-18': '% Federal Funding', 
                    '$ Instructional Expenditure per Pupil 2017-18':'$ Instructional Expenditure per Pupil'}, inplace=True)

In [42]:
financial18.head()

,District,District Size,% Local Property Taxes,% General State Aid,% Federal Funding,$ Instructional Expenditure per Pupil
3872,Payson CUSD 1,MEDIUM,45.3,34.9,7.4,4539.49
3873,Liberty CUSD 2,MEDIUM,41.1,35.4,5.5,5775.29
3874,Central CUSD 3,MEDIUM,39.7,39.3,9.0,5361.85
3875,CUSD 4,MEDIUM,49.3,33.9,5.6,4899.45
3876,Quincy SD 172,LARGE,53.0,20.8,12.0,6615.00


### Checking & renaming district name mismatches between datasets

In [43]:
# set(sy18_low_final['District'].unique()) - set(financial18['District'].unique())
# set(sy19_low_final['District'].unique()) - set(financial19['District'].unique())
# set(sy20_low_final['District'].unique()) - set(financial20['District'].unique())

In [44]:
#renaming districts if needed
sy18_low_final.at[2108, 'District'] = 'Oak Grove SD 68 Green Oaks'
sy18_low_final.at[2488, 'District'] = 'Oak Grove SD 68  Bartonville'

In [45]:
update = {'Ashton-Franklin Center CUSD 27': 'Ashton-Franklin Center CUSD 275',
          'Betty Shabazz  International C': 'Betty Shabazz  International Charter School',
          'Bluford Unit School District 3': 'Bluford Unit School District 318',
          'Bronzeville Academy Chtr Schoo': 'Bronzeville Academy Chtr School',
          'Chadwick-Milledgeville CUSD 39': 'Chadwick-Milledgeville CUSD 399',
          'Dimmick Community Consolidated': 'Dimmick Community Consolidated SD #175',
          'Elgin Math and Science Academy': 'Elgin Math and Science Academy Charter School',
          'Gibson City-Melvin-Sibley CUSD': 'Gibson City-Melvin-Sibley CUSD 5',
          'Horizon Science Acad-McKinley ': 'Horizon Science Acad-McKinley Park Charter Sch',
          'Horizon Science Acad-Belmont C': 'Horizon Science Acad-Belmont Charter Sch',
          'Huntley Community School Distr': 'Huntley Community School District 158',
          'Lincolnshire-Prairieview SD 10': 'Lincolnshire-Prairieview SD 103',
          'Maywood-Melrose Park-Broadview': 'Maywood-Melrose Park-Broadview 89',
          'Mercer County School District ': 'Mercer County School District 404',
          'Milford Area Public Schools Di': 'Milford Area Public Schools District 124',
          'N Pekin & Marquette Hght SD 10': 'N Pekin & Marquette Hght SD 102',
          'Norris City-Omaha-Enfield CUSD': 'Norris City-Omaha-Enfield CUSD 3',
          'Prairie Crossing Charter Schoo': 'Prairie Crossing Charter School',
          'Prophetstown-Lyndon-Tampico CU': 'Prophetstown-Lyndon-Tampico CUSD3',
          'Salt Fork Community Unit Distr': 'Salt Fork Community Unit District  512',
          'Spring Garden Community Consol': 'Spring Garden Community Consolidated School District 178',
          'Woodlawn Unit School District ': 'Woodlawn Unit School District 209'}
          
financial20.replace(update, inplace=True)

In [46]:
#merge with main df
sy18_low_final = sy18_low_final.merge(financial18, how = 'left', on = 'District')
sy19_low_final = sy19_low_final.merge(financial19, how = 'left', on = 'District')
sy20_low_final = sy20_low_final.merge(financial20, how = 'left', on = 'District')

### Merge final K-8 Datasets into 1 large df

In [47]:
#rearrange columns so all dfs are in the same order

order = ['key','RCDTS', 'NCES ID','school_year','Type', 'school_type2','School Name', 'District', 'City', 'County',
         'School Type', 'District Size', 'Grades Served','Summative Designation', 'State Senate District',
       'State Representative District', 'Charter', 'STITLEI', 'ULOCALE','Student Enrollment - Total',
       '% Student Enrollment - White',
       '% Student Enrollment - Black or African American',
       '% Student Enrollment - Hispanic or Latino',
       '% Student Enrollment - Asian',
       '% Student Enrollment - Native Hawaiian or Other Pacific Islander',
       '% Student Enrollment - American Indian or Alaska Native',
       '% Student Enrollment - Two or More Races', '% Student Enrollment - EL',
       '% Student Enrollment - IEP', '% Student Enrollment - Low Income',
       '% Student Enrollment - Homeless', 'Total Number of School Days',
       'Student Attendance Rate', 'Student Mobility Rate',
       'Student Mobility Rate - Male', 'Student Mobility Rate - Female',
       'Student Mobility Rate - White',
       'Student Mobility Rate - Black or African American',
       'Student Mobility Rate - Hispanic or Latino',
       'Student Mobility Rate - Asian',
       'Student Mobility Rate - Native Hawaiian or Other Pacific Islander',
       'Student Mobility Rate - American Indian or Alaska Native',
       'Student Mobility Rate - Two or More Races',
       'Student Mobility Rate - EL', 'Student Mobility Rate - IEP',
       'Student Mobility Rate - Low Income', 'Chronically Truant Students',
       'Student Chronic Truancy Rate', 'Avg Class Size – All Grades',
       'Teacher Retention Rate', 'Principal Turnover within 6 Years',
       'Chronic Absenteeism - White',
       'Chronic Absenteeism - Black or African American',
       'Chronic Absenteeism - Hispanic or Latino',
       'Chronic Absenteeism - Asian',
       'Chronic Absenteeism - Native Hawaiian or Other Pacific Islander',
       'Chronic Absenteeism - American Indian or Alaska Native',
       'Chronic Absenteeism - Two or More Races', 'Chronic Absenteeism - IEP',
       'Chronic Absenteeism - EL', 'Chronic Absenteeism - Low Income',
       'STUTERATIO', 'FTE',
       'IPR_EST', '% ELA Proficiency', '% ELA Proficiency - Low Income',
       '% Math Proficiency', '% Math Proficiency - Low Income', '% Local Property Taxes', '% General State Aid',
       '% Federal Funding', '$ Instructional Expenditure per Pupil', 'Total Incidents', 'LATCOD', 'LONCOD']

sy18_low_final = sy18_low_final[order]
sy19_low_final = sy19_low_final[order]
sy20_low_final = sy20_low_final[order]

In [48]:
# stack dataframes for combine K-8 dataset
df_lowerschool = pd.concat([sy18_low_final, sy19_low_final, sy20_low_final], axis=0)

#export to csv
df_lowerschool.to_csv('../Capstone/cleaned_datasets/cleaning/df_lowerschool.csv')

# Pre-Train Test Split Data Cleaning

In [49]:
df_k8 = df_lowerschool.copy()
df_k8.head(2)

,key,RCDTS,NCES ID,school_year,Type,school_type2,School Name,District,City,County,...,% ELA Proficiency - Low Income,% Math Proficiency,% Math Proficiency - Low Income,% Local Property Taxes,% General State Aid,% Federal Funding,$ Instructional Expenditure per Pupil,Total Incidents,LATCOD,LONCOD
0,Payson CUSD 1_Seymour Elementary School,010010010262002,173099003225,17-18,School,Lower School,Seymour Elementary School,Payson CUSD 1,Payson,Adams,...,26.1,16.9,9.7,45.3,34.9,7.4,4539.49,13.0,39.818244,-91.248138
1,Liberty CUSD 2_Liberty Elementary School,010010020262002,172277002523,17-18,School,Lower School,Liberty Elementary School,Liberty CUSD 2,Liberty,Adams,...,27.9,46.9,27.9,41.1,35.4,5.5,5775.29,16.0,39.886170,-91.108112


In [50]:
# Addressing Nulls that I can prior to the train-test-split - included in the function below
# pd.DataFrame(df_k8.isna().sum()).sort_values(by=0, ascending=False)

In [51]:
#renaming specific columns
df_k8.rename(columns = {'stitlei': 'Title I Status',
              'ulocale': 'Region Type',
              'ipr_est': 'inc_to_pov_ratio'}, inplace=True)

In [52]:
#create a function to clean dataset

def clean(df):
    #drop columns that I decided to not need or have lots of nulls
    df.drop(columns = ['District','School Name', 'City', 'Grades Served', 'Student Mobility Rate - Native Hawaiian or Other Pacific Islander',
                      'Chronic Absenteeism - Native Hawaiian or Other Pacific Islander',
                      'Chronic Absenteeism - American Indian or Alaska Native',
                      'Student Mobility Rate - American Indian or Alaska Native',
                       'key', 'Type', 'School Type', 'school_type2'], inplace=True)
    
    #cols to impute with 0
    c = ['% Student Enrollment - White', '% Student Enrollment - Black or African American', '% Student Enrollment - Hispanic or Latino',
         '% Student Enrollment - Asian', '% Student Enrollment - Native Hawaiian or Other Pacific Islander',
         '% Student Enrollment - American Indian or Alaska Native',
         '% Student Enrollment - Two or More Races',
         '% Student Enrollment - EL', '% Student Enrollment - IEP',
         '% Student Enrollment - Low Income', '% Student Enrollment - Homeless',
         'Student Mobility Rate - White', 'Student Mobility Rate - Black or African American',
         'Student Mobility Rate - Hispanic or Latino', 'Student Mobility Rate - Asian', 'Student Mobility Rate - Two or More Races',
         'Student Mobility Rate - EL', 'Student Mobility Rate - IEP',
         'Student Mobility Rate - Low Income', 'Chronic Absenteeism - White',
         'Chronic Absenteeism - Black or African American', 'Chronic Absenteeism - Hispanic or Latino',
         'Chronic Absenteeism - Asian','Chronic Absenteeism - Two or More Races',
         'Chronic Absenteeism - IEP', 'Chronic Absenteeism - EL', 'Chronic Absenteeism - Low Income']
    
    df[c] = df[c].replace(np.nan, 0)
    
    #impute NaNs in this categorical/ordinal to 'Missing' value
    df['STITLEI'] = df['STITLEI'].replace(np.nan, 'Missing')
    
    #convert ordinal variables into numeric
    lst = {'Exemplary': 4, 'Commendable': 3, 'Targeted': 2, 'Comprehensive': 1}
    
    for k, v in lst.items():
        df['Summative Designation'] = df['Summative Designation'].replace(k, v)
    
    #clean up column names
    df_k8.rename(columns = {'STITLEI': 'Title I Status',
              'ULOCALE': 'Region Type',
              'IPR_EST': 'inc_to_pov_ratio'}, inplace=True)
    
    df = df.rename(columns=str.lower)
    df.columns = df.columns.str.replace(' - ', ' ')
    df.columns = df.columns.str.replace(' – ', ' ') 
    df.columns = df.columns.str.replace('-', '')
    df.columns = df.columns.str.replace(' ', '_')
    
    
    return df

In [53]:
df_k8 = clean(df_k8)

In [54]:
#export df for preprocessing & feature engineering
# df_k8.to_csv('../Capstone/cleaned_datasets/pre-processing/df_k8.csv')